# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [ ]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [2]:
# Initialize and constants

load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-4o-mini'
openai = OpenAI()

API key looks good so far


In [ ]:
# A class to represent a Webpage

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"
    

In [ ]:
ed = Website("https://edwarddonner.com")
ed.links

['https://edwarddonner.com/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://patents.google.com/patent/US20210049536A1/',
 'https://www.linkedin.com/in/eddonner/',
 'https://edwarddonner.com/2024/11/13/llm-engineering-resources/',
 'https://edwarddonner.com/2024/11/13/llm-engineering-resources/',
 'https://edwarddonner.com/2024/10/16/from-software-engineer-to-ai-data-scientist-resources/',
 'https://edwarddonner.com/2024/10/16/from-software-engineer-to-ai-data-scientist-resources/',
 'https://edwarddonner.com/2024/08/06/outsmart/',
 'https://edwarddonner.com/2024/08/06/outsmart/',
 'https://edwarddonner.com/2024/06/26/choosing-the-right-llm-resources/

In [6]:
web = Website("https://udn.com/news/index")
web.links

['/news/index',
 '/news/index',
 'https://vip.udn.com/vip/index?utm_source=udn&utm_medium=referral&utm_campaign=familybar',
 'https://money.udn.com/money/index?utm_source=udn&utm_medium=referral&utm_campaign=familybar',
 'https://sdgs.udn.com/?utm_source=udn&utm_medium=referral&utm_campaign=familybar',
 'https://udesign.udnfunlife.com/mall/Cc1a00.do?sid=91_udn&utm_source=udn.com&utm_medium=referral_familybar_udn&utm_term=familybar_udn&utm_campaign=20170118_UDN',
 'https://utravel.udn.com/index.jsp?utm_source=udn&utm_medium=referral&utm_campaign=familybar',
 'https://visionproject.org.tw/?utm_source=udn&utm_medium=referral&utm_campaign=familybar',
 'https://udn.com/mypage/mynewsFeature?utm_source=udn&utm_medium=referral&utm_campaign=familybar',
 'https://vip.udn.com/',
 '//member.udn.com/member/center.jsp?hold=y',
 '//member.udn.com/member/ShowMember?actiontype=update',
 '//member.udn.com/member/order_list.jsp',
 '//member.udn.com/member/ProcessLogout',
 '/news/index',
 '/news/breaknews

## First step: Have GPT-4o-mini figure out which links are relevant

### Use a call to gpt-4o-mini to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [7]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""

In [8]:
print(link_system_prompt)

You are provided with a list of links found on a webpage. You are able to decide which of the links would be most relevant to include in a brochure about the company, such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}



In [9]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [10]:
print(get_links_user_prompt(ed))

Here is the list of links on the website of https://edwarddonner.com - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
https://edwarddonner.com/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://patents.google.com/patent/US20210049536A1/
https://www.linkedin.com/in/eddonner/
https://edwarddonner.com/2024/11/13/llm-engineering-resources/
https://edwarddonner.com/2024/11/13/llm-engineering-resources/
https://edwarddonner.com/2024/10/16/from-software-engineer-to-ai-data-scientist-resources/
https://edwarddonner

In [11]:
print(get_links_user_prompt(web))

Here is the list of links on the website of https://udn.com/news/index - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
/news/index
/news/index
https://vip.udn.com/vip/index?utm_source=udn&utm_medium=referral&utm_campaign=familybar
https://money.udn.com/money/index?utm_source=udn&utm_medium=referral&utm_campaign=familybar
https://sdgs.udn.com/?utm_source=udn&utm_medium=referral&utm_campaign=familybar
https://udesign.udnfunlife.com/mall/Cc1a00.do?sid=91_udn&utm_source=udn.com&utm_medium=referral_familybar_udn&utm_term=familybar_udn&utm_campaign=20170118_UDN
https://utravel.udn.com/index.jsp?utm_source=udn&utm_medium=referral&utm_campaign=familybar
https://visionproject.org.tw/?utm_source=udn&utm_medium=referral&utm_campaign=familybar
https://udn.com/mypage/mynewsFeature?utm_source=udn&utm_medium=referral&utm_c

In [ ]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)


In [13]:
# Anthropic has made their site harder to scrape, so I'm using HuggingFace..

huggingface = Website("https://huggingface.co")
huggingface.links

['/',
 '/models',
 '/datasets',
 '/spaces',
 '/posts',
 '/docs',
 '/enterprise',
 '/pricing',
 '/login',
 '/join',
 '/meta-llama/Llama-3.3-70B-Instruct',
 '/nesaorg/benchmark_v0',
 '/tencent/HunyuanVideo',
 '/black-forest-labs/FLUX.1-dev',
 '/Qwen/QwQ-32B-Preview',
 '/models',
 '/spaces/JeffreyXiang/TRELLIS',
 '/spaces/ginipick/FLUXllama',
 '/spaces/Yuanshi/OminiControl',
 '/spaces/Kwai-Kolors/Kolors-Virtual-Try-On',
 '/spaces/multimodalart/flux-style-shaping',
 '/spaces',
 '/datasets/HuggingFaceFW/fineweb-2',
 '/datasets/fka/awesome-chatgpt-prompts',
 '/datasets/CohereForAI/Global-MMLU',
 '/datasets/O1-OPEN/OpenO1-SFT',
 '/datasets/foursquare/fsq-os-places',
 '/datasets',
 '/join',
 '/pricing#endpoints',
 '/pricing#spaces',
 '/pricing',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/allenai',
 '/facebook',
 '/amazon',
 '/google',
 '/Intel',
 '/microsoft',
 '/grammarly',
 '/Writer',
 '/docs/transformers',
 '/docs/diffu

In [14]:
get_links("https://huggingface.co")

{'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'},
  {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'},
  {'type': 'company page', 'url': 'https://huggingface.co/enterprise'},
  {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'},
  {'type': 'blog page', 'url': 'https://huggingface.co/blog'},
  {'type': 'community page', 'url': 'https://discuss.huggingface.co'}]}

In [15]:
get_links("https://udn.com/news/index")

{'links': [{'type': 'about page', 'url': 'https://www.udngroup.com/'},
  {'type': 'careers page', 'url': 'https://co.udn.com/co/contactus'},
  {'type': 'news page', 'url': 'https://udn.com/news/index'},
  {'type': 'company page', 'url': 'https://udn.com/mypage/mynewsFeature'}]}

## Second step: make the brochure!

Assemble all the details into another prompt to GPT4-o

In [ ]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result


In [17]:
print(get_all_details("https://huggingface.co"))

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'blog', 'url': 'https://huggingface.co/blog'}, {'type': 'community page', 'url': 'https://discuss.huggingface.co'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}
Landing page:
Webpage Title:
Hugging Face – The AI community building the future.
Webpage Contents:
Hugging Face
Models
Datasets
Spaces
Posts
Docs
Enterprise
Pricing
Log In
Sign Up
The AI community building the future.
The platform where the machine learning community collaborates on models, datasets, and applications.
Trending on
this week
Model

In [26]:
print(get_all_details("https://udn.com/news/index"))

Found links: {'links': [{'type': 'company page', 'url': 'https://www.udngroup.com/'}, {'type': 'careers page', 'url': 'https://member.udn.com/member/center.jsp?hold=y'}, {'type': 'about page', 'url': 'https://udn.com/news/index'}, {'type': 'contact page', 'url': 'https://co.udn.com/co/contactus'}, {'type': 'events page', 'url': 'https://event.udn.com/index/index.html'}]}
Landing page:
Webpage Title:
聯合新聞網：觸動未來新識力
Webpage Contents:
聯合報
經濟日報
陽光行動
有設計
有行旅
願景
我的新聞
會員中心
我的設定
訂閱管理
登出
即時
要聞
娛樂
運動
全球
社會
地方
產經
股市
房市
生活
寵物
健康
橘世代
文教
評論
兩岸
科技
Oops
閱讀
旅遊
雜誌
報時光
倡議+
500輯
轉角國際
NBA
時尚
汽車
棒球
HBL
遊戲
專題
網誌
女子漾
購物
全產品速覽
新聞評論
聯合新聞網
陽光行動
聯合報數位版
聯合70
轉角國際
倡議家
世界日報
新聞話題
服務
會員中心
U利點數
我的新聞
歷史新聞
活動專區
udn粉絲團
udn line 好友
股市理財
經濟日報網
房地產
中經社
樂透
發票
生活娛樂
噓！星聞
優人物
udnSTYLE
500輯
遊戲角落
發燒車訊
元氣網
橘世代
寵物部落
科技玩家
女子漾
報時光
運動
udn NBA
野球夢田
HBL
閱讀創作
琅琅悅讀
琅琅書店
讀創故事
一刻鯨選
聯副70
udn部落格
聯合文學
聯經出版
聯文雜誌
文創購物
數位文創
IP 授權
瘋活動
售票網
時光商號
更多產品
聯合知識庫
聯合電子報
聯合影音網
聯合學苑
有行旅
APP行動網
內容聯盟
關於我們
P
訂閱
刊登廣告
FAQ
·
客服
新聞授權
服務條款
·
著作權
·
隱私權聲明
聯合報系
訂報紙
關於我們
網

In [37]:
# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."


In [29]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [20]:
get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'blog', 'url': 'https://huggingface.co/blog'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}]}


"You are looking at a company called: HuggingFace\nHere are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\nLanding page:\nWebpage Title:\nHugging Face – The AI community building the future.\nWebpage Contents:\nHugging Face\nModels\nDatasets\nSpaces\nPosts\nDocs\nEnterprise\nPricing\nLog In\nSign Up\nThe AI community building the future.\nThe platform where the machine learning community collaborates on models, datasets, and applications.\nTrending on\nthis week\nModels\nmeta-llama/Llama-3.3-70B-Instruct\nUpdated\n1 day ago\n•\n59.2k\n•\n882\nnesaorg/benchmark_v0\nUpdated\n7 days ago\n•\n173M\n•\n670\ntencent/HunyuanVideo\nUpdated\n5 days ago\n•\n2.94k\n•\n936\nblack-forest-labs/FLUX.1-dev\nUpdated\nAug 16\n•\n1.37M\n•\n7.17k\nQwen/QwQ-32B-Preview\nUpdated\n13 days ago\n•\n84.7k\n•\n1.24k\nBrowse 400k+ models\nSpaces\nRunning\non\nZero\n968\n🏢\nTRELLIS\nScalable and Versatile 3D Generation from imag

In [30]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [22]:
create_brochure("HuggingFace", "https://huggingface.com")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.com/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'enterprise page', 'url': 'https://huggingface.com/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.com/pricing'}, {'type': 'blog page', 'url': 'https://huggingface.com/blog'}, {'type': 'community page', 'url': 'https://discuss.huggingface.co'}, {'type': 'github page', 'url': 'https://github.com/huggingface'}, {'type': 'linkedin page', 'url': 'https://www.linkedin.com/company/huggingface/'}, {'type': 'twitter page', 'url': 'https://twitter.com/huggingface'}]}


# Hugging Face Brochure

**The AI Community Building the Future**

---

## About Us

Hugging Face is an innovative leader in the machine learning community, dedicated to democratizing AI technologies. Our mission is to make machine learning accessible and effective for everyone, fostering collaboration and creativity among developers, researchers, and organizations. Our platform is a thriving hub for sharing models, datasets, and applications, designed to support open-source initiatives and enhance user experiences.

---

## Our Features

### **Collaboration Platform**
- **Models & Datasets**: Browse and collaborate on over 400k models and 100k datasets.
- **Spaces**: Develop and showcase applications in various modalities such as text, image, video, and audio.
  
### **Enterprise Solutions**
- Tailored support with enterprise-grade security and access controls for teams looking to leverage AI effectively.
- Pricing starting at **$20/user/month** for dedicated support and advanced features.

### **Open Source Initiatives**
- Hugging Face champions open-source development with libraries like **Transformers**, **Diffusers**, and **Tokenizers**, fostering contributions and improvements from our global community.

---

## Our Customers

More than **50,000 organizations** rely on Hugging Face for their machine-learning needs, including industry giants like:
- Amazon Web Services
- Google
- Microsoft
- Intel
- Meta

These organizations leverage our tools and community resources to advance their AI projects and innovate in their fields.

---

## Company Culture

At Hugging Face, we believe in a collaborative, inclusive workplace where creativity thrives. With a growing team of over **225 experts**, we embrace diversity and welcome everyone to contribute towards our mission. Our culture promotes ongoing learning and community engagement, and our work environment is centered around support, growth, and mutual respect.

---

## Join Us

If you are passionate about advancing machine learning, we want you to be a part of our journey. Explore career opportunities at Hugging Face and help us shape the future of AI.

- **Explore Opportunities**: [Jobs at Hugging Face](https://huggingface.co/jobs)

---

## Connect with Us

Stay connected with Hugging Face through our social media channels for updates, community stories, and more.

- **GitHub**: [Hugging Face GitHub](https://github.com/huggingface)
- **Twitter**: [@huggingface](https://twitter.com/huggingface)
- **LinkedIn**: [Hugging Face LinkedIn](https://www.linkedin.com/company/huggingface)
- **Discord**: Join our Discord community for discussions and collaborations!

---

**Thank you for your interest in Hugging Face! Together, we are building the future of AI.**

In [27]:
create_brochure("Microsoft", "https://www.microsoft.com")

Found links: {'links': [{'type': 'about page', 'url': 'https://www.microsoft.com/en-us/about'}, {'type': 'careers page', 'url': 'https://careers.microsoft.com'}, {'type': 'company information', 'url': 'https://www.microsoft.com/en-us/investor/default.aspx'}, {'type': 'diversity page', 'url': 'https://www.microsoft.com/en-us/diversity/'}, {'type': 'sustainability page', 'url': 'https://www.microsoft.com/en-us/sustainability/'}, {'type': 'accessibility page', 'url': 'https://www.microsoft.com/en-us/accessibility'}]}


# Microsoft Brochure

## About Microsoft
Microsoft is a global technology leader renowned for its innovative software, hardware, and services. Founded in 1975, Microsoft has evolved into one of the most influential companies in the tech industry, shaping how individuals and businesses interact with technology. The company is dedicated to empowering every person and every organization on the planet to achieve more.

## Product Offerings
- **Microsoft 365**: A comprehensive suite of productivity tools including Word, Excel, PowerPoint, and Teams for enhanced collaboration.
- **Windows**: The operating system that powers billions of devices worldwide, offering a user-friendly interface and robust security.
- **Surface Devices**: A range of touch-enabled PCs designed for versatility, performance, and mobility.
- **Xbox**: A leader in gaming consoles and services, providing immersive entertainment experiences with titles available through Xbox Game Pass and Xbox Live Gold.
- **Azure**: Microsoft’s cloud computing service designed to help businesses build, manage, and deploy applications in the cloud.

## Company Culture
At Microsoft, inclusivity, collaboration, and innovation are at the heart of the company’s culture. Microsoft prioritizes diverse perspectives and fosters a supportive environment where every employee is encouraged to grow and contribute to meaningful projects. The company champions work-life balance and offers numerous resources to support employee well-being.

### Values:
- **Innovation**: Constantly pushing the boundaries of technology to deliver cutting-edge solutions.
- **Diversity**: Embracing a workforce as diverse as its global customer base.
- **Integrity**: Upholding the highest standards of ethical practices in all business dealings.

## Customers
Microsoft serves a wide range of customers, from individual consumers and small businesses to large enterprises and governments. Its products are utilized by millions of users across various sectors, ensuring that technology is accessible, reliable, and powerful enough to meet diverse needs.

## Careers at Microsoft
Microsoft offers a dynamic work environment with opportunities for growth, development, and impact. The company actively seeks talented individuals who are eager to bring their unique skills and perspectives to the table. With various roles across software development, marketing, sales, and support, Microsoft is dedicated to helping employees build fulfilling careers.

### Employee Benefits:
- Competitive salaries and bonuses
- Health and wellness programs
- Professional development and training opportunities
- Work-from-home options and flexible schedules

## Join Us
Whether you’re a prospective customer looking for innovative solutions, an investor interested in a tech giant with a committed vision, or a recruit eager to join a diverse and dynamic team, Microsoft welcomes you to embark on a journey of impact and inspiration.

For more information, visit [microsoft.com](https://www.microsoft.com).

---

This brochure provides a snapshot of Microsoft’s offerings, culture, and career opportunities, reflecting the company's dedication to innovation and its mission to empower people and organizations worldwide.

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [38]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)


In [35]:
stream_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}]}



# Hugging Face: The AI Community Building the Future

## About Us
Hugging Face is a leading collaborative platform for the machine learning community, where individuals and organizations come together to create, discover, and collaborate on machine learning models, datasets, and applications. Our mission is to make machine learning accessible and beneficial for everyone, from researchers and developers to enterprises worldwide.

## What We Offer
- **Models:** Access a vast array of over 400,000 models including state-of-the-art options such as Llama-3.3-70B.   
- **Datasets:** Discover and share 100,000+ datasets designed for various tasks in machine learning.  
- **Spaces:** Engage with over 150,000 applications and experiment with cutting-edge technology like FLUX for style transfer and Kolors for virtual try-ons.  
- **Enterprise Solutions:** Tailored packages that offer enhanced security, advanced features, and dedicated support for teams at an affordable price.

## Our Community
- More than **50,000 organizations** use Hugging Face, including industry giants like Meta, Amazon Web Services, Google, and Microsoft.
- Join a vibrant community of machine learning practitioners who collaborate on groundbreaking projects and innovations daily.

## Company Culture
At Hugging Face, we believe in fostering an open-source environment where creativity and collaboration thrive. Our culture is defined by:
- **Inclusivity:** We welcome and celebrate diverse backgrounds and ideas. 
- **Innovation:** Encouragement for team members to experiment and push the boundaries of what's possible in AI.
- **Community-Driven:** As we build the future of AI, community feedback is vital in shaping our tools and resources.

## Careers and Opportunities
We are always on the lookout for talented individuals passionate about machine learning and AI! At Hugging Face, you will have the chance to work in a dynamic environment with a focus on personal growth and career development. Join us to:
- Make a tangible impact in the AI landscape.
- Collaborate with leading experts in the field.
- Contribute to pioneering projects and open-source initiatives.

Explore current job openings and become part of our mission to democratize AI technology!

## Connect with Us
Stay updated and engage with our community:
- **Website:** [HuggingFace.com](https://huggingface.co)
- **Social Media:** Follow us on [Twitter](https://twitter.com/huggingface), [LinkedIn](https://www.linkedin.com/company/huggingface), and [Discord](https://discord.gg/huggingface).

Join us in building the future of AI!



In [39]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

stream_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'docs page', 'url': 'https://huggingface.co/docs'}, {'type': 'discussion forum', 'url': 'https://discuss.huggingface.co'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}]}


# 🌟 Welcome to Hugging Face! 🌟  
*The AI community building the future – one cozy hug at a time!*

---

## 🤖 What We Are All About  
At Hugging Face, we believe that machine learning should be as friendly as a warm hug and as easy to use as a Sunday morning coffee. Our platform connects a lively community where you can **create**, **discover**, and **collaborate** on **AI models**, **datasets**, and **applications**. 

---

## 🛠️ What We Offer  
- **🎉 Over 400k models** - From llama-themed algorithms to cutting-edge video AI, we've got something for everyone!  
- **📊 100k+ datasets** - Ready to dive into the world of data? Bring your swimsuit!  
- **🚀 Apps Galore** - Want to see how you look in a digital jacket? Try our *Virtual Try-On*! 🧥  
- **🌌 Spaces** - Create and run your own unique AI experience! Imagine a universe where you are the overlord of AI… or just someone testing out the latest features.  

---

## 🏢 Company Culture  
At Hugging Face, we're like a family gathering at Thanksgiving: passionate discussions, joyous laughter, and occasionally someone dropping the mashed potatoes (not metaphorically, but we’ll clean it up!). We value collaboration, creativity, and the occasional dad joke. Y'know, the good stuff!

- **Diversity and Inclusion:** We embrace diverse perspectives because different ideas make better AI (and more entertaining office banter).
- **Open Source Spirit:** We’re building the foundation of machine learning tooling with the community—think of it like a potluck where everyone contributes a dish. You build, we eat! 🍽️

---

## 💼 Careers at Hugging Face  
Want to join us in this delightful ride? We’re on the lookout for enthusiastic minds to help us **democratize* good ML! If you’re looking for a place that values innovation and the occasional TikTok dance-off, check out our [jobs page](https://huggingface.co) - we promise, it’s more fun than cleaning your cat's litter box. 

---

## 🎉 Who Loves Us?  
Join the ranks of **more than 50,000 organizations** like:
- **Meta, Google, and Microsoft** (yes, the cool kids)
- **Grammarly** - because what good is AI if it can't help you avoid typos in your love letters? ❤️

---

## 📣 Let's Connect!  
Join our *hugging community* on Discord, Twitter, or LinkedIn! Together, let's build the future - and maybe squeeze in a few group hugs along the way. 

---

**Remember, whatever your AI dreams are, we’re just a hug away!** 🤗  

*Join today, your friendly AI adventure awaits!*

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype.</span>
        </td>
    </tr>
</table>